In [2]:
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [3]:
def clean_text(text):
    return re.sub(r'\s+', ' ', text.strip().replace('\u202f', ''))

In [4]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

In [5]:
data = []

for page in range(1, 100):
    url = f"https://dakar-auto.com/senegal/location-de-voitures-19?page={page}"
    print(f"🔎 Scraping page {page}...")

    try:
        driver.get(url)
        time.sleep(2)

        annonces = driver.find_elements(By.CSS_SELECTOR, ".listings-cards__list-item")

        for annonce in annonces:
            try:
                # ✅ V1 : marque & V2 : année
                titre = clean_text(annonce.find_element(By.CSS_SELECTOR, "h2 a").text)
                parts = titre.split()
                marque = parts[0] if len(parts) > 0 else ""
                annee = parts[-1] if parts[-1].isdigit() and len(parts[-1]) == 4 else ""

                # ✅ V3 : prix
                prix = clean_text(annonce.find_element(By.CSS_SELECTOR, "h3").text)
                prix = prix.replace("F CFA", "").replace(" ", "")

                # ✅ V4 : adresse (ville + région)
                ville = ""
                region = ""
                try:
                    ville = clean_text(annonce.find_element(By.CLASS_NAME, "town-suburb").text)
                except:
                    pass
                try:
                    region = clean_text(annonce.find_element(By.CLASS_NAME, "province").text)
                except:
                    pass
                adresse = f"{ville}{region}".strip(",")

                # ✅ V5 : propriétaire (sans "Par ")
                try:
                    proprio = clean_text(annonce.find_element(By.CLASS_NAME, "time-author").text.replace("Par ", ""))
                except:
                    proprio = ""

                data.append({
                    "marque": marque,
                    "annee": annee,
                    "prix (FCFA)": prix,
                    "adresse": adresse,
                    "proprietaire": proprio
                })

            except Exception as e:
                print(f"⚠️ Erreur dans une annonce : {e}")
                continue

    except Exception as e:
        print(f"❌ Erreur chargement page {page} : {e}")
        continue

driver.quit()

🔎 Scraping page 1...
🔎 Scraping page 2...
🔎 Scraping page 3...
🔎 Scraping page 4...
🔎 Scraping page 5...
🔎 Scraping page 6...
🔎 Scraping page 7...
🔎 Scraping page 8...
🔎 Scraping page 9...
🔎 Scraping page 10...
🔎 Scraping page 11...
🔎 Scraping page 12...
🔎 Scraping page 13...
🔎 Scraping page 14...
🔎 Scraping page 15...
🔎 Scraping page 16...
🔎 Scraping page 17...
🔎 Scraping page 18...
🔎 Scraping page 19...
🔎 Scraping page 20...
🔎 Scraping page 21...
🔎 Scraping page 22...
🔎 Scraping page 23...
🔎 Scraping page 24...
🔎 Scraping page 25...
🔎 Scraping page 26...
🔎 Scraping page 27...
🔎 Scraping page 28...
🔎 Scraping page 29...
🔎 Scraping page 30...
🔎 Scraping page 31...
🔎 Scraping page 32...
🔎 Scraping page 33...
🔎 Scraping page 34...
🔎 Scraping page 35...
🔎 Scraping page 36...
🔎 Scraping page 37...
🔎 Scraping page 38...
🔎 Scraping page 39...
🔎 Scraping page 40...
🔎 Scraping page 41...
🔎 Scraping page 42...
🔎 Scraping page 43...
🔎 Scraping page 44...
🔎 Scraping page 45...
🔎 Scraping page 46.

In [9]:
df = pd.DataFrame(data)
pd.set_option("display.max_colwidth", None)
df.head(1000)

,marque,annee,prix (FCFA),adresse,proprietaire
0,Citroen,2014,20000,"Médina,Dakar",Chasseur Automobile
1,Peugeot,2013,30000,"Grand-Yoff,Dakar",Chic Drive
2,Ford,2014,30000,"Sangalkam,Dakar",Medoune Ndiaye
3,Ford,2015,35000,"Cité Keur gorgui,Dakar",Pape Cheikh automobile
4,Toyota,2020,35000,"Mermoz Sacré-Coeur,Dakar",LocAuto Sénégal Location de voiture
...,...,...,...,...,...
151,Hyundai,2016,20000,"Yoff,Dakar",A vos services
152,Dodge,2012,30000,"Yoff,Dakar",A vos services
153,Hyundai,2017,35000,"Yoff,Dakar",Abdia Sn
154,Ford,2012,40000,"Sicap Liberté,Dakar",KINGCARS Service


In [8]:
df.to_csv("locations_clean.csv", index=False, encoding="utf-8-sig")
